In [1]:
import numpy as np
import tensorflow as tf

C:\Users\MengnanChen\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [21]:
sess=tf.InteractiveSession()

In [ ]:
'''
refer to: https://blog.csdn.net/huachao1001/article/details/79120521
'''

In [ ]:
'''对于in_c个通道的输入图，如果需要通过卷积后输出out_c个feature map，
那么总共需要in_c*out_c个卷积核'''

In [4]:
#输入，shape=[c,h,w]，此处shape=[2,5,5]
input_data=np.asarray([
              [[1,0,1,2,1],
               [0,2,1,0,1],
               [1,1,0,2,0],
               [2,2,1,1,0],
               [2,0,1,2,0]],

               [[2,0,2,1,1],
                [0,1,0,0,2],
                [1,0,0,2,1],
                [1,1,2,1,0],
                [1,0,1,1,1]],
])

In [5]:
input_data.shape

(2, 5, 5)

In [6]:
#卷积核，shape=[in_c,k,k]=[2,3,3]
weights_data=np.asarray([ 
               [[ 1, 0, 1],
                [-1, 1, 0],
                [ 0,-1, 0]],
               [[-1, 0, 1],
                [ 0, 0, 1],
                [ 1, 1, 1]] 
             ])

In [7]:
weights_data.shape

(2, 3, 3)

In [11]:
c,h,w=input_data.shape
cols=[]
for i in range(c):
    line=np.reshape(input_data[i],[h*w,1])
    cols.append(line)

In [12]:
cols[0]

array([[1],
       [0],
       [1],
       [2],
       [1],
       [0],
       [2],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [2],
       [0],
       [2],
       [2],
       [1],
       [1],
       [0],
       [2],
       [0],
       [1],
       [2],
       [0]])

In [15]:
cols[1]

array([[2],
       [0],
       [2],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [2],
       [1],
       [0],
       [0],
       [2],
       [1],
       [1],
       [1],
       [2],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1]])

In [14]:
np.concatenate(cols,axis=1)

array([[1, 2],
       [0, 0],
       [1, 2],
       [2, 1],
       [1, 1],
       [0, 0],
       [2, 1],
       [1, 0],
       [0, 0],
       [1, 2],
       [1, 1],
       [1, 0],
       [0, 0],
       [2, 2],
       [0, 1],
       [2, 1],
       [2, 1],
       [1, 2],
       [1, 1],
       [0, 0],
       [2, 1],
       [0, 0],
       [1, 1],
       [2, 1],
       [0, 1]])

In [17]:
# shape: [h,w,c]
np.reshape(np.concatenate(cols,axis=1),[h,w,c])

array([[[1, 2],
        [0, 0],
        [1, 2],
        [2, 1],
        [1, 1]],

       [[0, 0],
        [2, 1],
        [1, 0],
        [0, 0],
        [1, 2]],

       [[1, 1],
        [1, 0],
        [0, 0],
        [2, 2],
        [0, 1]],

       [[2, 1],
        [2, 1],
        [1, 2],
        [1, 1],
        [0, 0]],

       [[2, 1],
        [0, 0],
        [1, 1],
        [2, 1],
        [0, 1]]])

In [8]:
def get_shape(tensor):
    s1,s2,s3=tensor.get_shape()
    s1,s2,s3=int(s1),int(s2),int(s3)
    return s1,s2,s3

In [9]:
def chw2hwc(chw_tensor):
    '''
    Tensorflow中定义的Tensor的shape为[n,h,w,c]，在这里将n设为1，batch_size设为1，
    刚刚将输入定义为[c,h,w]，因此需要将[c,h,w]转为[h,w,c]
    '''
    c,h,w=get_shape(chw_tensor)
    cols=[]
    
    for i in range(c):
        line=tf.reshape(chw_tensor[i],[h*w,1])
        cols.append(line)
    
    # [h*w,c]
    inputs=tf.concat(cols,axis=1)
    inputs=tf.reshape(inputs,[h,w,c])
    return inputs

In [ ]:
'''
Tensorflow使用卷积核时，使用的格式是[k,k,in_c,out_c]，
上述定义卷积核时使用的格式是[in_c,k,k]。
这里需要将[in_c,k,k] -> [k,k,in_c]。简化工作起见，设置out_c=1.
因此可以直接对卷积核调用chw2hwc，再在最后一个维度上扩充即可形成[k,k,in_c,out_c]
'''

In [18]:
def hwc2chw(hwc_tensor):
    h,w,c=get_shape(hwc_tensor)
    cs=[]
    for i in range(c):
        # [h,w]->[1,h,w]
        channel=tf.expand_dims(hwc_tensor[:,:,i],axis=0)
        cs.append(channel)
    # [1,h,w]...[1,h,w]->[c,h,w]
    inputs=tf.concat(cs,axis=0)
    return inputs

In [19]:
def tf_conv2d(inputs,weights):
    conv=tf.nn.conv2d(inputs,weights,strides=[1,1,1,1],padding='SAME')
    return conv

In [20]:
const_input=tf.constant(input_data,tf.float32)
const_weights=tf.constant(weights_data,tf.float32)

In [22]:
sess.run(const_input)

array([[[1., 0., 1., 2., 1.],
        [0., 2., 1., 0., 1.],
        [1., 1., 0., 2., 0.],
        [2., 2., 1., 1., 0.],
        [2., 0., 1., 2., 0.]],

       [[2., 0., 2., 1., 1.],
        [0., 1., 0., 0., 2.],
        [1., 0., 0., 2., 1.],
        [1., 1., 2., 1., 0.],
        [1., 0., 1., 1., 1.]]], dtype=float32)

In [23]:
sess.run(const_weights)

array([[[ 1.,  0.,  1.],
        [-1.,  1.,  0.],
        [ 0., -1.,  0.]],

       [[-1.,  0.,  1.],
        [ 0.,  0.,  1.],
        [ 1.,  1.,  1.]]], dtype=float32)

In [24]:
inputs=tf.Variable(const_input,name='inputs')
# [c,h,w]->[h,w,c], here: [2,5,5]->[5,5,2]
inputs=chw2hwc(inputs)
# [h,w,c]->[batch_size,h,w,c], here: [5,5,2]->[1,5,5,2]
inputs=tf.expand_dims(inputs,axis=0)

In [25]:
weights=tf.Variable(const_weights,name='weights')
# [in_c,k,k]->[k,k,in_c], here: [2,3,3]->[3,3,2]
weights=chw2hwc(weights)
# [k,k,in_c]->[k,k,in_c,out_c], here: [3,3,2,1]
weights=tf.expand_dims(weights,axis=3)

In [26]:
# [batch_size,h,w,c]
conv=tf.nn.conv2d(inputs,weights,strides=[1,1,1,1],padding='SAME')

In [28]:
init=tf.global_variables_initializer()
sess.run(init)
# [batch_size,h,w,c]
sess.run(conv)

array([[[[ 2.],
         [ 0.],
         [ 2.],
         [ 4.],
         [ 0.]],

        [[ 1.],
         [ 4.],
         [ 4.],
         [ 3.],
         [ 5.]],

        [[ 4.],
         [ 3.],
         [ 5.],
         [ 9.],
         [-1.]],

        [[ 3.],
         [ 4.],
         [ 6.],
         [ 2.],
         [ 1.]],

        [[ 5.],
         [ 3.],
         [ 5.],
         [ 1.],
         [-2.]]]], dtype=float32)

In [29]:
rst=hwc2chw(conv[0])

In [30]:
# [c,h,w]
sess.run(rst)

array([[[ 2.,  0.,  2.,  4.,  0.],
        [ 1.,  4.,  4.,  3.,  5.],
        [ 4.,  3.,  5.,  9., -1.],
        [ 3.,  4.,  6.,  2.,  1.],
        [ 5.,  3.,  5.,  1., -2.]]], dtype=float32)

In [31]:
###########################使用numpy实现conv2d###########################

input_data=[
              [[1,0,1,2,1],
               [0,2,1,0,1],
               [1,1,0,2,0],
               [2,2,1,1,0],
               [2,0,1,2,0]],

               [[2,0,2,1,1],
                [0,1,0,0,2],
                [1,0,0,2,1],
                [1,1,2,1,0],
                [1,0,1,1,1]] 
]

weights_data=[ 
               [[ 1, 0, 1],
                [-1, 1, 0],
                [ 0,-1, 0]],
               [[-1, 0, 1],
                [ 0, 0, 1],
                [ 1, 1, 1]] 
]

In [34]:
print(np.asarray(input_data).shape)
print(np.asarray(weights_data).shape)

(2, 5, 5)
(2, 3, 3)


In [35]:
x=[
    [1,2,3],
    [4,5,6],
    [7,8,9]
]
y=[
    [1,2,3],
    [4,5,6],
    [7,8,9]
]
x=np.asarray(x)
y=np.asarray(y)

In [36]:
x*y

array([[ 1,  4,  9],
       [16, 25, 36],
       [49, 64, 81]])

In [38]:
np.sum(x*y)

285

In [43]:
def compute_conv(feature_map,kernel):
    '''
    feature map: [h,w]
    kernel: [k,k]
    return rs: [h,w]
    输出一个卷积核组中的一个卷积核对一个通道中的feature map进行卷积后的结果
    '''
    h,w=feature_map.shape
    k,_=kernel.shape
    r=int(k/2)
    # 定义填充0之后的feature map
    padding_fm=np.zeros([h+2,w+2],np.float32)
    # 保存计算结果
    rst=np.zeros([h,w],np.float32)
    # 将输入赋值到指定区域，即除了0填充的边界，剩下的区域
    padding_fm[1:h+1,1:w+1]=feature_map
    
    # 以每个点为中心进行遍历，获得每一个通道上卷积核对feature map的卷积
    # 该结果与其余通道的结果元素加，获得该组卷积核的feature map
    for i in range(1,h+1):
        for j in range(1,w+1):
            roi=padding_fm[i-r:i+r+1,j-r:j+r+1]
            rst[i-1][j-1]=np.sum(roi*kernel)
            
    return rst

In [48]:
def my_conv2d(inputs,weights):
    c,h,w=inputs.shape
    _,k,_=weights.shape  # [in_c,k,k]
    outputs=[]
    out_c=1  # 输出通道数设置为1
    
    for _ in range(out_c):
        output_=np.zeros([h,w],np.float32)

        for i in range(c):
            # 当前通道feature map: [h,w]
            feature_map=inputs[i]
            # 当前通道kernel: [k,k]
            w=weights[i]
            rst=compute_conv(feature_map,w)
            output_=output_+rst
            
        # 对out_c组卷积核执行卷积操作    
        outputs.append(output_)
    return outputs

In [49]:
inputs=np.asarray(input_data,np.float32)
weights=np.asarray(weights_data,np.float32)
rst=my_conv2d(inputs,weights)

In [50]:
print(rst)

[array([[ 2.,  0.,  2.,  4.,  0.],
       [ 1.,  4.,  4.,  3.,  5.],
       [ 4.,  3.,  5.,  9., -1.],
       [ 3.,  4.,  6.,  2.,  1.],
       [ 5.,  3.,  5.,  1., -2.]], dtype=float32)]


In [ ]:
##############################MobileNet##############################
'''
refer to: https://blog.csdn.net/huachao1001/article/details/79171447

MobileNet是针对移动端的优化，当需要压缩模型时，可以考虑使用MobileNet代替卷积

相比于普通卷积，MobileNet采用的方法是，将卷积分解为两个操作：depthwise和pointwise
pointwise：即普通的1*1卷积
depthwise：
- 普通的卷积：对于输入通道数为in_channel的feature map，在计算卷积时，
输出的每个通道都需要对应in_channel个filter_size*filter_size卷积核参数。
- MobileNet把每个输入feature map对应一个卷积核，输出通道数不变。
真正对通道数做改变的是pointwise，也即是1*1卷积。
'''

In [ ]:
'''
Tensorflow中的depthwise操作，pointwise操作就是普通的conv1d

tf.nn.depthwise_conv2d(
    input,
    filter,
    strides,
    padding,
    rate=None,
    name=None,
    data_format=None
)

tf.nn.conv1d(
    value,
    filters,
    stride,
    padding,
    use_cudnn_on_gpu=None,
    data_format=None,
    name=None
)
'''

In [51]:
# 输入，shape=[c,h,w]=[2,5,5]
input_data=[
              [[1,0,1,2,1],
               [0,2,1,0,1],
               [1,1,0,2,0],
               [2,2,1,1,0],
               [2,0,1,2,0]],

               [[2,0,2,1,1],
                [0,1,0,0,2],
                [1,0,0,2,1],
                [1,1,2,1,0],
                [1,0,1,1,1]],
            ]

# 卷积核，shape=[in_c,k,k]=[2,3,3]
weights_data=[ 
               [[ 1, 0, 1],
                [-1, 1, 0],
                [ 0,-1, 0]],
               [[-1, 0, 1],
                [ 0, 0, 1],
                [ 1, 1, 1]] 
             ]

In [52]:
def get_shape(tensor):
    s1,s2,s3=tensor.get_shape()
    s1,s2,s3=int(s1),int(s2),int(s3)
    return s1,s2,s3

def chw2hwc(chw_tensor):
    c,h,w=get_shape(chw_tensor)
    cols=[]
    
    for i in range(c):
        line=tf.reshape(chw_tensor[i],[h*w,1])
        cols.append(line)
        
    # [h*w,c]
    inputs=tf.concat(cols,axis=-1)
    # [h*w,c]->[h,w,c]
    inputs=tf.reshape(inputs,[h,w,c])
    return inputs

def hwc2chw(hwc_tensor):
    h,w,c=get_shape(hwc_tensor)
    cs=[]
    for i in range(c):
        # get [h,w]->[1,h,w] 
        channel=tf.expand_dims(hwc_tensor[:,:,i],axis=0)
        cs.append(channel)
    
    # [1,h,w]...[1,h,w]->[c,h,w]
    inputs=tf.concat(cs,axis=0)
    return inputs

In [54]:
const_inputs=tf.constant(input_data,tf.float32)
const_weights=tf.constant(weights_data,tf.float32)

# [2,5,5]
inputs=tf.Variable(const_inputs,name='input')
# [2,5,5]->[5,5,2]
inputs=chw2hwc(inputs)
# [5,5,2]->[1,5,5,2]
inputs=tf.expand_dims(inputs,axis=0)

# [2,3,3]
weights=tf.Variable(const_weights,name='weights')
# [2,3,3]->[3,3,2]
weights=chw2hwc(weights)
[3,3,2,1]
weights=tf.expand_dims(weights,axis=3)

conv=tf.nn.depthwise_conv2d(inputs,filter=weights,strides=[1,1,1,1],padding='SAME')
rst=hwc2chw(conv[0])

In [55]:
init=tf.global_variables_initializer()
sess.run(init)

In [56]:
sess.run(rst)

array([[[ 1., -3.,  0.,  1., -2.],
        [-1.,  3.,  1., -1.,  3.],
        [ 1., -1.,  0.,  3., -2.],
        [ 1.,  1.,  1., -2.,  1.],
        [ 4.,  1.,  4.,  2., -1.]],

       [[ 1.,  3.,  2.,  3.,  2.],
        [ 2.,  1.,  3.,  4.,  2.],
        [ 3.,  4.,  5.,  6.,  1.],
        [ 2.,  3.,  5.,  4.,  0.],
        [ 1.,  2.,  1., -1., -1.]]], dtype=float32)

In [57]:
input_data=[
              [[1,0,1,2,1],
               [0,2,1,0,1],
               [1,1,0,2,0],
               [2,2,1,1,0],
               [2,0,1,2,0]],

               [[2,0,2,1,1],
                [0,1,0,0,2],
                [1,0,0,2,1],
                [1,1,2,1,0],
                [1,0,1,1,1]] 
]

weights_data=[ 
               [[ 1, 0, 1],
                [-1, 1, 0],
                [ 0,-1, 0]],
               [[-1, 0, 1],
                [ 0, 0, 1],
                [ 1, 1, 1]] 

]

In [65]:
def compute_conv(feature_map,kernel):
    h,w=feature_map.shape
    k,_=kernel.shape
    r=int(k/2)
    # 初始化填充后的map
    padding_fm=np.zeros([h+2,w+2],np.float32)
    # 保存计算结果
    rst=np.zeros([h,w],np.float32)
    # 将输入赋值到指定区域，即除了0填充的边界，剩下的区域
    padding_fm[1:h+1,1:w+1]=feature_map
    for i in range(1,h+1):  # i: 1~h
        for j in range(1,w+1):  # j: 1~w
            # 取出以当前点为中心的k*k区域
            roi=padding_fm[i-r:i+r+1,j-r:j+r+1]
            # 计算当前点的卷积，对k*k个点，点乘之后求和
            rst[i-1][j-1]=np.sum(roi*kernel)
        
    return rst

def my_depthwise(chw_inputs,chw_weights):
    c,_,_=chw_inputs.shape
    _,k,_=chw_weights.shape
    outputs=[]
    
    for i in range(c):
        # [h,w]
        feature_map=chw_inputs[i]
        # [k,k]
        weight=chw_weights[i]
        rst=compute_conv(feature_map,weight)
        # 注意：和conv2d不同，此处不再是元素加，
        # 而是将每一个通道卷积后的结果直接保存下来
        outputs.append(rst)
        
    return np.asarray(outputs)

In [66]:
inputs=np.asarray(input_data,np.float32)
weights=np.asarray(weights_data,np.float32)
rst=my_depthwise(inputs,weights)

In [67]:
rst

array([[[ 1., -3.,  0.,  1., -2.],
        [-1.,  3.,  1., -1.,  3.],
        [ 1., -1.,  0.,  3., -2.],
        [ 1.,  1.,  1., -2.,  1.],
        [ 4.,  1.,  4.,  2., -1.]],

       [[ 1.,  3.,  2.,  3.,  2.],
        [ 2.,  1.,  3.,  4.,  2.],
        [ 3.,  4.,  5.,  6.,  1.],
        [ 2.,  3.,  5.,  4.,  0.],
        [ 1.,  2.,  1., -1., -1.]]], dtype=float32)

In [ ]:
#######################反卷积#######################
'''
反卷积：反卷积后，下一层feature map的尺寸比上一层大。
在卷积过程中，先将feature map内部以0填充，扩大尺寸，然后执行通常的卷积

refer to: https://blog.csdn.net/huachao1001/article/details/79131814
'''

In [69]:
input_data=[
               [[1,0,1],
                [0,2,1],
                [1,1,0]],

               [[2,0,2],
                [0,1,0],
                [1,0,0]],

               [[1,1,1],
                [2,2,0],
                [1,1,1]],

               [[1,1,2],
                [1,0,1],
                [0,2,2]]

            ]
weights_data=[ 
              [[[ 1, 0, 1],
                [-1, 1, 0],
                [ 0,-1, 0]],
               [[-1, 0, 1],
                [ 0, 0, 1],
                [ 1, 1, 1]],
               [[ 0, 1, 1],
                [ 2, 0, 1],
                [ 1, 2, 1]], 
               [[ 1, 1, 1],
                [ 0, 2, 1],
                [ 1, 0, 1]]],

              [[[ 1, 0, 2],
                [-2, 1, 1],
                [ 1,-1, 0]],
               [[-1, 0, 1],
                [-1, 2, 1],
                [ 1, 1, 1]],
               [[ 0, 0, 0],
                [ 2, 2, 1],
                [ 1,-1, 1]], 
               [[ 2, 1, 1],
                [ 0,-1, 1],
                [ 1, 1, 1]]]  
           ]

In [72]:
'''
每个卷积核需要旋转180°再传入tf.nn.conv2d_transpose函数，因为tf.nn.conv2d_transpose内部
会旋转180°，所以提前旋转再经过内部旋转，能够保证卷积核和我们使用的卷积核的数据排列一致

我们定义的卷积核shape为[out_c,in_c,h,w]需要转化为Tensorflow反卷积中使用的[h,w,out_c,in_c]
'''
weights_np=np.asarray(weights_data,np.float32)
# 将输入的每个卷积核旋转180°
weights_np=np.rot90(weights_np,k=2,axes=(2,3))

const_input=tf.constant(input_data,tf.float32)
const_weights=tf.constant(weights_np,tf.float32)

inputs=tf.Variable(const_input,tf.float32)
const_weights=tf.constant(weights_np,tf.float32)

inputs=tf.Variable(const_input,name='inputs')
# [c,h,w]->[h,w,c]
inputs=tf.transpose(inputs,perm=[1,2,0])
# [h,w,c]->[batch_size,h,w,c]
inputs=tf.expand_dims(inputs,axis=0)

# [out_c,in_c,h,w]
weights=tf.Variable(const_weights,name='weights')
# [out_c,in_c,h,w]->[h,w,out_c,in_c]
weights=tf.transpose(weights,perm=(2,3,0,1))

#######################Tensorflow反卷积#######################
# input_shape=[batch_size,h,w,c]
input_shape=inputs.get_shape().as_list()
# weights_shape=[h,w,out_c,in_c]
weights_shape=weights.get_shape().as_list()
output_shape=[input_shape[0],input_shape[1]*2,input_shape[2]*2,
              weights_shape[2]]
deconv=tf.nn.conv2d_transpose(inputs,weights,output_shape=output_shape,
                             strides=[1,2,2,1],padding='SAME')

In [73]:
init=tf.global_variables_initializer()
sess.run(init)
deconv_val=sess.run(deconv)

In [89]:
'''
在TF中，Conv2d-CPU模式下目前仅支持NHWC格式，即[Number, Height, Weight, Channel]格式。
Conv2d-GPU模式下以NCHW为主，但支持将NHWC转换为NCHW求解。
下面的是Conv2d-CPU即NHWC格式
'''

deconv_val[0]  # batch 1

array([[[ 4.,  4.],
        [ 3.,  1.],
        [ 6.,  7.],
        [ 2.,  0.],
        [ 7.,  7.],
        [ 3.,  2.]],

       [[ 4.,  5.],
        [ 3.,  6.],
        [ 3.,  0.],
        [ 2.,  1.],
        [ 7.,  8.],
        [ 5.,  5.]],

       [[ 8.,  8.],
        [ 6.,  0.],
        [ 8.,  8.],
        [ 5., -2.],
        [11., 14.],
        [ 2.,  2.]],

       [[ 3.,  3.],
        [ 2.,  3.],
        [ 7.,  9.],
        [ 2.,  8.],
        [ 3.,  1.],
        [ 3.,  0.]],

       [[ 5.,  3.],
        [ 5.,  0.],
        [11., 13.],
        [ 3.,  0.],
        [ 9., 11.],
        [ 3.,  2.]],

       [[ 2.,  3.],
        [ 1.,  5.],
        [ 4.,  3.],
        [ 5.,  1.],
        [ 4.,  3.],
        [ 4.,  0.]]], dtype=float32)

In [92]:
# nhwc->nchw
sess.run(tf.transpose(deconv_val,perm=(0,3,1,2)))

array([[[[ 4.,  3.,  6.,  2.,  7.,  3.],
         [ 4.,  3.,  3.,  2.,  7.,  5.],
         [ 8.,  6.,  8.,  5., 11.,  2.],
         [ 3.,  2.,  7.,  2.,  3.,  3.],
         [ 5.,  5., 11.,  3.,  9.,  3.],
         [ 2.,  1.,  4.,  5.,  4.,  4.]],

        [[ 4.,  1.,  7.,  0.,  7.,  2.],
         [ 5.,  6.,  0.,  1.,  8.,  5.],
         [ 8.,  0.,  8., -2., 14.,  2.],
         [ 3.,  3.,  9.,  8.,  1.,  0.],
         [ 3.,  0., 13.,  0., 11.,  2.],
         [ 3.,  5.,  3.,  1.,  3.,  0.]]]], dtype=float32)

In [85]:
#####################numpy实现反卷积#####################

def compute_conv(feature_map,kernel):
    h,w=feature_map.shape
    k,_=kernel.shape
    r=int(k/2)
    padding_fm=np.zeros([h+2,w+2],np.float32)
    rst=np.zeros([h,w],np.float32)
    padding_fm[1:h+1,1:w+1]=feature_map
    for i in range(1,h+1):
        for j in range(1,w+1):
            roi=padding_fm[i-r:i+r+1,j-r:j+r+1]
            rst[i-1][j-1]=np.sum(roi*kernel)
    return rst

def fill_zeros(inputs):
    '''
    内部填充0
    '''
    c,h,w=inputs.shape
    rst=np.zeros([c,2*h+1,2*w+1],np.float32)
    
    for i in range(c):
        for j in range(h):
            for k in range(w):
                rst[i,2*j+1,2*k+1]=inputs[i,j,k]
    
    return rst

def my_deconv(inputs,weights):
    out_c,in_c,h,w=weights.shape
    out_h=2*h
    out_w=2*w
    outputs=[]
    
    for i in range(out_c):
        w=weights[i]
        output_=np.zeros([out_h,out_w],np.float32)
        for j in range(in_c):
            conv=compute_conv(inputs[j],w[j])
            # 裁剪
            output_=output_+conv[0:out_h,0:out_w]
        outputs.append(output_)
        
    return outputs

In [86]:
inputs=np.asarray(input_data,np.float32)
inputs=fill_zeros(inputs)
weights=np.asarray(weights_data,np.float32)
deconv=my_deconv(inputs,weights)

In [87]:
'''
注意，下面是NCHW格式
'''

deconv

[array([[ 4.,  3.,  6.,  2.,  7.,  3.],
        [ 4.,  3.,  3.,  2.,  7.,  5.],
        [ 8.,  6.,  8.,  5., 11.,  2.],
        [ 3.,  2.,  7.,  2.,  3.,  3.],
        [ 5.,  5., 11.,  3.,  9.,  3.],
        [ 2.,  1.,  4.,  5.,  4.,  4.]], dtype=float32),
 array([[ 4.,  1.,  7.,  0.,  7.,  2.],
        [ 5.,  6.,  0.,  1.,  8.,  5.],
        [ 8.,  0.,  8., -2., 14.,  2.],
        [ 3.,  3.,  9.,  8.,  1.,  0.],
        [ 3.,  0., 13.,  0., 11.,  2.],
        [ 3.,  5.,  3.,  1.,  3.,  0.]], dtype=float32)]

In [141]:
input_data2=np.zeros((10,10)).astype(np.float32)
for x in range(4,7):
    for y in range(3,6):
        input_data2[y,x] = 1
        
weights_data2=np.array([
    [1,2,3],
    [4,5,6],
    [7,8,9]
]).astype(np.float32)

In [148]:
tf_dilateconv = tf.nn.atrous_conv2d(np.expand_dims(np.expand_dims(input_data2,axis=0),axis=3),
                                    np.expand_dims(np.expand_dims(np.rot90(weights_data2,2),axis=3),axis=4),
                                    rate=2,padding="VALID")
sess.run(tf.squeeze(tf_dilateconv))

C:\Users\MengnanChen\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  


array([[ 1.,  1.,  3.,  2.,  5.,  3.],
       [ 5.,  5., 12.,  7., 16.,  9.],
       [ 4.,  4.,  9.,  5., 11.,  6.],
       [11., 11., 24., 13., 28., 15.],
       [ 7.,  7., 15.,  8., 17.,  9.],
       [ 7.,  7., 15.,  8., 17.,  9.]], dtype=float32)

In [128]:
from scipy.signal import convolve2d

def my_dialateconv2d(inputs,weights):
    in_c,h,w=inputs.shape
    out_c,in_c,k,_=weights.shape
    outputs=[]
    
    for i in range(out_c):
        for j in range(in_c):
            output_=None
            kernel=weights[i][j]
            input_data=inputs[j]
            for t in range(k*2-1):
                if t%2!=0:
                    # 向卷积核中内部补0，注意：在实际操作中，并不增加卷积核的参数数量
                    kernel=np.insert(kernel,t,values=0.,axis=0)
                    kernel=np.insert(kernel,t,values=0.,axis=1)
            
            conved=convolve2d(input_data,kernel,mode='valid')
            output_=conved if output_ is None else conved+output_
             
        outputs.append(output_)
    return outputs

In [144]:
inputs=np.expand_dims(np.asarray(input_data2,np.float32),axis=0)
weights=np.expand_dims(np.expand_dims(np.asarray(weights_data2,np.float32),axis=0),axis=0)
my_dilateconv=my_dialateconv2d(inputs,weights)

In [145]:
my_dilateconv

[array([[ 1.,  1.,  3.,  2.,  5.,  3.],
        [ 5.,  5., 12.,  7., 16.,  9.],
        [ 4.,  4.,  9.,  5., 11.,  6.],
        [11., 11., 24., 13., 28., 15.],
        [ 7.,  7., 15.,  8., 17.,  9.],
        [ 7.,  7., 15.,  8., 17.,  9.]], dtype=float32)]